In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

In [ ]:
df = pd.read_csv("14-income_evaluation.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
col_names = ["age", "workclass", "finalweight", "education", "education_num", "marital_status", "occupation", 
            "relationship", "race", "sex","capital_gain", "capital_loss","hours_per_week", "native_country","income"]

In [ ]:
df.columns = col_names

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
categorical = [col for col in df.columns if df[col].dtype == "O"]
numerical = [col for col in df.columns if df[col].dtype != "O"]

In [ ]:
categorical

In [ ]:
numerical

In [ ]:
df[categorical].head()

In [ ]:
for col in categorical: 
    print(df[col].value_counts())

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
ax = sns.countplot(x = "income", hue = "sex", data = df)
ax.set_title("Distribution of income by gender")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
ax = sns.countplot(x = "income", hue = "race", data = df)
ax.set_title("Distribution of income by gender")
plt.show()

In [ ]:
sns.catplot(y = df["hours_per_week"], hue = df["income"])
plt.show()

In [ ]:
over_40_hours = df[df["hours_per_week"] > 40]
under_40_hours = df[df["hours_per_week"] <= 40]

In [ ]:
over_40_hours["income"].value_counts()

In [ ]:
under_40_hours["income"].value_counts()

In [ ]:
df["workclass"].unique()

In [ ]:
df["workclass"] = df["workclass"].replace(" ?", np.nan)

In [ ]:
df["workclass"].value_counts()

In [ ]:
df["marital_status"].unique()

In [ ]:
df["occupation"].unique()

In [ ]:
df["occupation"] = df["occupation"].replace(" ?", np.nan)

In [ ]:
df["native_country"] = df["native_country"].replace(" ?", np.nan)

In [ ]:
df.isnull().sum()

In [ ]:
sns.pairplot(df, hue = "income")
plt.show()

In [ ]:
X = df.drop("income", axis = 1)
y = df["income"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [ ]:
categorical = [col for col in X_train.columns if X_train[col].dtype == "O"]

In [ ]:
X_train[categorical]

In [ ]:
X_train[categorical].isnull().sum()

In [ ]:
X_test[categorical].isnull().sum()

In [ ]:
for i in [X_train, X_test]: 
    i["workclass"] = i["workclass"].fillna(X_train["workclass"].mode()[0])
    i["occupation"] = i["occupation"].fillna(X_train["occupation"].mode()[0])
    i["native_country"] = i["native_country"].fillna(X_train["native_country"].mode()[0])

In [ ]:
X_train[categorical].isnull().sum()

In [ ]:
X_test[categorical].isnull().sum()

In [ ]:
# encoding

In [ ]:
X_train[categorical].head()

In [ ]:
df[categorical].nunique()

In [ ]:
y_train_binary = y_train.apply(lambda x: 1 if x.strip() == ">50K" else 0)

In [ ]:
target_means = y_train_binary.groupby(X_train["native_country"]).mean()

In [ ]:
X_train["native_country_encoded"] = X_train["native_country"].map(target_means)
X_train["native_country_encoded"] = X_train["native_country_encoded"].fillna(y_train_binary.mean())

X_test["native_country_encoded"] = X_test["native_country"].map(target_means)
X_test["native_country_encoded"] = X_test["native_country_encoded"].fillna(y_train_binary.mean())

In [ ]:
X_train.head()

In [ ]:
X_train.drop("native_country", axis = 1, inplace = True)
X_test.drop("native_country", axis = 1, inplace = True)

In [ ]:
categorical

In [ ]:
one_hot_categories = ['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [ ]:
encoder = ColumnTransformer(
    transformers = [
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), one_hot_categories)
    ], remainder = "passthrough"
)

In [ ]:
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

In [ ]:
columns = encoder.get_feature_names_out()

In [ ]:
columns

In [ ]:
X_train = pd.DataFrame(X_train_enc, columns = columns, index = X_train.index)
X_test = pd.DataFrame(X_test_enc, columns = columns, index = X_test.index)

In [ ]:
X_train

In [ ]:
cols = X_train.columns

In [ ]:
from sklearn.preprocessing import RobustScaler 
scaler = RobustScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train, columns = cols)
X_test = pd.DataFrame(X_test, columns = cols)

In [ ]:
# training 

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, random_state=15)
rfc.fit(X_train, y_train)

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=15)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
feature_scores =pd.Series(rfc.feature_importances_, index = X_train.columns).sort_values(ascending=False)

In [ ]:
feature_scores

In [ ]:
feature_scores.tail(10)

In [ ]:
X_train = X_train.drop(["cat__education_ 12th", "cat__race_ Other", "cat__education_ 5th-6th", "cat__education_ 1st-4th", "cat__marital_status_ Married-AF-spouse", "cat__occupation_ Priv-house-serv",
                        "cat__workclass_ Without-pay", "cat__education_ Preschool", "cat__occupation_ Armed-Forces", "cat__workclass_ Never-worked"], axis = 1)

In [ ]:
X_train.head()

In [ ]:
X_test = X_test.drop(["cat__education_ 12th", "cat__race_ Other", "cat__education_ 5th-6th", "cat__education_ 1st-4th", "cat__marital_status_ Married-AF-spouse", "cat__occupation_ Priv-house-serv",
                        "cat__workclass_ Without-pay", "cat__education_ Preschool", "cat__occupation_ Armed-Forces", "cat__workclass_ Never-worked"], axis = 1)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=15)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# hyperparameter tuning 

In [ ]:
rfc_params = {
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [5,8,10,15, None], 
    "max_features": ["sqrt", "log2", 5,6,7,8],
    "min_samples_split": [2,8,15,20]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV 
rfc = RandomForestClassifier()

In [ ]:
rscv = RandomizedSearchCV(estimator=rfc, param_distributions=rfc_params, cv = 3, n_jobs = -1)
rscv.fit(X_train, y_train)

In [ ]:
y_pred = rscv.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
rscv.best_params_